# BIP Framework tuning example 

This process is **NOT** taken from the original paper, but it is a way to tune the parameters of the BIP Framework.

In [ ]:
using Pkg
Pkg.activate("..")
using BIPs

In [ ]:
using Statistics
using Pkg.Artifacts

Lets begin by bringing in the dataset. It contains tree splits:
* **train**: the training set with 1M jets
* **validation**: the validation set with 400k jets

And of course later we will use the **test** set with other 400k jets to report the results


In [ ]:
dataset_path = "../../../DataLake/raw"
train_data_path = dataset_path*"/train.h5"
val_data_path = dataset_path*"/val.h5"

### Reading the data

In order to read the datasets, we call the `read_dataset` function:
to read the TopQuark format

In [ ]:
train_jets, train_labels = BIPs.read_data("TQ", train_data_path)
train_labels = [reinterpret(Bool, b == 1.0) for b in train_labels]
print("Number of entries in the training data: ", length(train_jets))

In [ ]:
val_jets, val_labels = BIPs.read_data("TQ", val_data_path)
val_labels = [reinterpret(Bool, b == 1.0) for b in val_labels]
print("Number of entries in the validation data: ", length(val_jets))

Lets examine how one of the jets looks like, each one of the entries is one detected particle's four momentum $(E, p_x, p_y, p_z)$.

However,in order to compute the embeddings, it is necesary to convert the jets to a format that can be used by the framework. The function `data2hyp` allows to convert each detected four momentum to the jet basis, a.k.a $(\tilde p_T, \cos(\theta), \sin(\theta), \tilde y, E_T)$

In [ ]:
train_transf_jets = data2hyp(train_jets)
val_transf_jets = data2hyp(val_jets)
println("Transformed jets")

### The embeddings

Once the jets are converted to the jet basis, it is moment to embed the model using the *Invariant Polynomials*. 

The function `build_ip` allocates efficiently the sparse basis, while the `bip_data` computes the invariant representation of each one of the jets.

In [ ]:
f_bip, specs = build_ip(order=4, levels=8)
    
function bip_data(dataset_jets)
    storage = zeros(length(dataset_jets), length(specs))
    for i = 1:length(dataset_jets)
        storage[i, :] = f_bip(dataset_jets[i])
    end
    storage[:, 2:end]
end

In [ ]:
train_embedded_jets = bip_data(train_transf_jets)
println("Embedded train jets correclty")
val_embedded_jets = bip_data(val_transf_jets)
println("Embedded test jets correclty")

### Training a classifier model

The embeddings are now created for the dataset. From this point on, the classification itself is absolutelly versatile. For this specific example we will use the out-of-the box classifier `sklearn.linear_model.HistGradientBoostingClassifier` that bines the data and then applies a grandient boosted trees algorithm. 

Now, lets fit a simple model to the data.


In [ ]:
length(specs)

In [ ]:
using PyCall
@pyimport sklearn.neural_network as sk_nn

In [ ]:
mlp = sk_nn.MLPClassifier(verbose=true,max_iter=2000, njobs=-1)

In [ ]:
parameter_space = Dict(
    "hidden_layer_sizes"  => [ (100,), (50,50,50), (200,100,50,25), (100,50)],
    "activation"  => ["tanh", "relu", "logistic"],
)

In [ ]:
@pyimport sklearn.model_selection as sk_ms

In [ ]:
clf = sk_ms.GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)


In [ ]:
clf.fit(train_embedded_jets, train_labels)

In [ ]:
clf

# Lest test how we do performance

Now that we understanad the framework, lets see how our model performs on the test set.

In [ ]:
test_data_path = "../../../DataLake/raw/test.h5"

In [ ]:
test_jets, test_labels = BIPs.read_data("TQ", test_data_path)
test_labels = [reinterpret(Bool, b == 1.0) for b in test_labels]
test_transf_jets = data2hyp(test_jets)
test_embedded_jets = bip_data(test_transf_jets)
print("Embedded test jets correclty")

In [ ]:
test_preds = clf.score(test_embedded_jets, test_labels)

In [ ]:
using Plots
test_probas = clf.predict_proba(test_embedded_jets)
bkg_index = [label==false for label in test_labels]
Plots.histogram(test_probas[:, 2][test_labels], color="Blue", label="Top Jets", xlabel="Model's Output Probability", ylabel="Number of Jets")
Plots.histogram!(test_probas[:, 2][bkg_index], color="Red", label="QCD Jets", title="Probability Scores")